<a href="https://colab.research.google.com/github/mmrmas/SRAMicrobiomeCategoryPipeline/blob/sam/HMP_data_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# HMP data training for model


In [240]:
!pip install -Uqq fastbook waterfallcharts treeinterpreter dtreeviz #kaggle 
!pip install -q unpackai

In [241]:
from fastbook           import *
from unpackai.tabular   import no_missing_values, plot_hist

from pandas.api.types   import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble   import RandomForestRegressor
from sklearn.ensemble   import RandomForestClassifier
from sklearn.tree       import DecisionTreeRegressor
from dtreeviz.trees     import *
from IPython.display    import Image, display_svg, SVG

import seaborn as sns

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8


**Machine Learning Task**:  Categorize micrbiome profiles based on their origin (from the HMP dataset) 

**Target**: categories

**Samples**: data samples from individuals

**Features**: taxon 

**Values**: frequency data of the microbiome, will be normalized against "Bacteria"

#### 1. Prepare the dataset

Download the file from dropbox, where features are taxa, disease indiction and tissue source. 

In [242]:
df = pd.read_csv("https://www.dropbox.com/s/jgeae300bbjpzm5/HMPFeatureTable.csv?dl=1", sep=',')

Explore the dataset to choose the column name for setting a target variable.

One could select a particular tissue source to dive into

In [243]:
#select the tissue
#df = df[df['categories'] == 'HMP_ForegutEsophagealAdenocarcinoma']
#df = df[df['affected'] != 'Yes-Normal'] 

In [ ]:
df

At the moment the target feature still has to be createdd. It will be a concatenation of the tissue origin and the disease indication

In [245]:
targetFeature = 'affected_categories'                                            
#removeFeature = 'affected'

first get rid of the columns that are not used (categories or  affected)

In [ ]:
pd.set_option('display.max_columns', None)
df['affected_categories'] = df['categories'] + '_' + df['affected'].apply(str)  
target = [targetFeature]  
df = df.drop(['categories', 'affected'],axis=1)      
#df = df.drop([removeFeature],axis=1)               #include this for instance when only 'categories' are calssified             
df[targetFeature]

In [ ]:
#check our categories and show how many we have of each
categories = list(set(df[target]))
size_per_category = pd.pivot_table(df, index = targetFeature, values = 'Bacteria', aggfunc = [len])  
list(size_per_category.index.values)
graph_df = pd.DataFrame({'lab':list(size_per_category.index.values), 'count':list(size_per_category.iloc[0:,0])})
ax = graph_df.plot.bar(x='lab', y='count', rot=90)

In [ ]:
categories = list(set(df[target]))
categories

In [ ]:
size_per_category 

In [ ]:
categs = list(size_per_category.iloc[0:,0])
categs.sort()
minval = int(categs[0])
print (minval)

At the moment let's include categories with more then 76 observations

In [ ]:
minval = 76
#need to get equal number of categories
slimmed_down = list()
for s in list(size_per_category.index.values):
  this_sample = df[df[targetFeature]==s]               
  if (len(this_sample) > minval-1):
    l = list(range(len(this_sample)))
    include = random.sample(l, minval)
    include_this = this_sample.iloc[include]
    slimmed_down.append(include_this)
df = pd.concat(slimmed_down)
df

Here we normalize all values against the "Bacteria" taxon in order to align samples of differnt origin better, and to be able to export and to be able to re-use the  model as a .pt model later

In [252]:
#divide through "Bacteria" value
targetCol = df[target]
dfDrop = df.drop(target,axis=1) #remove the 'category' column

In [253]:
df = dfDrop.iloc[:,:].div(df.Bacteria, axis=0) #divide everything by the bacteria value 

In [254]:
df = dfDrop.join(targetCol) #bring back the target

Utilize `cont_cat_split` to derive continious and categorival variables (not explicitely needed in this type of datasets though)

In [255]:
cont_var, cat_var = cont_cat_split(df, dep_var=target)

#### 2. Data Transformations

In [ ]:
cat_var #just checkin'!

In [ ]:
splits=RandomSplitter(valid_pct=0.3)(df)
splits

In [ ]:
df = no_missing_values(df, 0.6)

In [259]:
df = df.fillna(0) #get rid of missing values

Setting up the Random Forest. Parameters can probably be changed

In [ ]:
#@title Random Forest Functionality

def rf_feat_importance(xs, y, valid_xs, valid_y):
    """Fit a RF model on the passed xs and y, and evaluate oob and validation R2 score"""
    print("Setting up model ...")
    n_estimators = 40
    max_samples = min(50_000, xs.shape[0])
    max_features = 0.5
    min_samples_leaf = 5
    m = RandomForestClassifier(n_estimators=n_estimators, 
                              max_samples=max_samples, 
                              max_features=max_features, 
                              min_samples_leaf=min_samples_leaf, 
                              oob_score=True,
                              n_jobs=-1,
                              random_state=88)

    print("Fitting model ...")
    m.fit(X=xs, y=y)
    oob_r2 = m.oob_score_
    val_r2 = m.score(valid_xs, valid_y)

    return pd.DataFrame({'cols':xs.columns, 'imp':m.feature_importances_}
                        ).sort_values('imp', ascending=True)

def get_oob_R2(xs, y, c=[]):
    """Fits a RF model and evaluate a OOB score on xs where features c are dropped
    params: 
        xs: the training set
        y:  the targets for training
        c:  (optional): the name of one column or a list of columns to drop from xs
    """
    if isinstance_str(c, 'str'): c = [c]
    if set(c).difference(set(xs.columns)) != set():
        return None
    n_estimators = 40
    max_samples = min(50_000, xs.shape[0])
    max_features = 0.5
    min_samples_leaf = 5
    m = RandomForestClassifier(n_estimators=n_estimators, 
                              max_samples=max_samples, 
                              max_features=max_features, 
                              min_samples_leaf=min_samples_leaf, 
                              oob_score=True,
                              n_jobs=-1,
                              random_state=88)
    m.fit(X=xs.drop(c, axis=1), y=y)
    oob_r2 = m.oob_score_
    #print(f"OOB R2 Score:")
    return oob_r2

#Before we can run the random forest, we also need to define the DataBlock.
to = TabularPandas(df, 
                   procs=[Categorify, FillMissing, Normalize],
                   cat_names=cat_var,
                   cont_names=cont_var,
                   y_names=target,
                   splits=splits)

xs, y = to.train.xs, to.train.y #training set
valid_xs, valid_y = to.valid.xs, to.valid.y #validation set

get_oob_R2(xs, y)

In [ ]:
#let's look at the feature importance
fi = rf_feat_importance(xs, y, valid_xs, valid_y)
fi[:]
#len(fi)

keep only those above a certain threshold

In [ ]:
to_keep = fi[fi.imp>0.0001].cols
xs_imp = xs[to_keep]
valid_xs_imp = valid_xs[to_keep]
to_keep

In [ ]:
get_oob_R2(xs_imp, y)
#miraculously this may DEPEND ON THE ORDER of the data...?

In [ ]:
cluster_columns(xs_imp, figsize=(12, 20))

In [ ]:
# Try to remove features that reduce the oob
# simply remove the feature that leads to maximal oob improvement when it is removed
# probably not perfect to reach global oob minimum but hopefully reasonable to improve the feature selection

#frst set the status quo
prev_oob = get_oob_R2(xs_imp, y)

while (1==1): #loop untill the new oob value does not increase any longer by removing features
  #loop through the fetaures and remove the feature with the lowest oob if it is lower than the new oob
  new_oobs = []
  for c in (cat_var):
    temp_oob = get_oob_R2(xs_imp, y, c)
    if (temp_oob != None):
      new_oobs.append([c , temp_oob])
  for c in (cont_var):
    temp_oob = get_oob_R2(xs_imp, y, c)
    if (temp_oob != None):
      new_oobs.append([c , temp_oob])

  #check if the lowest value can be removed: only when it makes the oob higher
  new_oobs = sorted(new_oobs,key=lambda x: (x[1]), reverse= True)
  new_oob_value = new_oobs[0][1]

  print(f"new OOB: {new_oob_value} from feature {new_oobs[0][0]} old OOB: {prev_oob}")
  if (new_oob_value  > prev_oob): #clean up the features
    xs_imp = xs_imp.drop(labels = new_oobs[0][0], axis = 1) 
    valid_xs_imp = valid_xs_imp.drop(labels = new_oobs[0][0], axis = 1)
    prev_oob = new_oob_value
  else:
    break  




In [ ]:
cluster_columns(xs_imp, figsize=(12, 8))

In [267]:
keys = xs_imp.keys()
cont_var_final, cat_var_final  = cont_cat_split(xs[keys])

In [ ]:
cont_var_final

In [269]:
to = TabularPandas(df, 
                   procs=[Categorify,FillMissing,Normalize],
                   cont_names=cont_var_final,
                   y_names=target,
                   y_block=CategoryBlock,
                   splits=splits)

xs_final, y = to.train.xs, to.train.y
valid_xs_final, valid_y = to.valid.xs, to.valid.y

#dls = to.dataloaders(bs=1024)
dls = to.dataloaders()

#### 4. Model Training

In [270]:
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
splits

In [ ]:
learn.fit_one_cycle(20)

#### 5. Interpret the model and make predictions

In [ ]:
outcome = learn.show_results(dl=dls.valid, max_n=20)
outcome

In [ ]:
row, clas, probs = learn.predict(df.iloc[2])
print(probs)
print(learn.dls.vocab)
row.show()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
# confusion matrix as an array
print(interp.confusion_matrix())

# confusion as a plot
interp.plot_confusion_matrix( figsize=(12, 12))

store

In [276]:
# Save the entire model as a pkl (to re-use in this script) and pt (to use as a standalone off-line).
!mkdir -p drive/MyDrive/saved_model/
learn.export('drive/MyDrive/saved_model/Model.pkl') 
pd.Series(xs_imp.keys()).to_csv('drive/MyDrive/saved_model/xses.csv', index=False, header=False) 